In [408]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
data = pd.read_csv('assignment_2.csv')
data.head()

,Label,bias,experience,salary
0,1,1,0.7,48000
1,0,1,1.9,48000
2,1,1,2.5,60000
3,0,1,4.2,63000
4,0,1,6.0,76000


In [413]:
bias = data["bias"]
Label = data["Label"]
scaler = StandardScaler() # Scaling
df_new = pd.DataFrame(scaler.fit_transform(data), columns = data.columns)
df_new["bias"] = bias
df_new["Label"] = Label
df_new

,Label,bias,experience,salary
0,1,1,-1.513249,-1.206602
1,0,1,-1.090751,-1.206602
2,1,1,-0.879501,-0.432312
3,0,1,-0.280962,-0.238739
4,0,1,0.352786,0.600075
...,...,...,...,...
195,0,1,0.528827,1.116268
196,0,1,0.669660,0.406502
197,0,1,0.035912,0.341978
198,1,1,1.444241,0.148406


In [392]:
dataX_bias

0      1
1      1
2      1
3      1
4      1
      ..
195    1
196    1
197    1
198    1
199    1
Name: bias, Length: 200, dtype: int64

## sigmoid

In [106]:
import random

In [645]:
y = np.array(df_new["Label"])
x = np.array(df_new[["bias", "experience", "salary"]])
beta = np.array([random.random(), random.random(), random.random()])
beta

array([0.19992772, 0.01903386, 0.85982076])

In [599]:
def sigmoid(x, beta) :
    multiplier = 0
    for i in range(x.size):
        multiplier += x[i]*beta[i]
    p = 1.0/(1.0+np.exp(-multiplier))
    return p
sigmoid(x[0], beta)

0.17631891826681273

## log likelihood

In [639]:
def lg_likelihood(x, y, beta) :
    log_p_hat = 0
    for i in range(y.size) :
        p = sigmoid(x[i], beta)
        log_p_hat += y[i]*np.log(p) + (1-y[i])*np.log(1-p)
        
    return log_p_hat
lg_likelihood(x, y, beta)

-201.37205325530977

In [601]:
#개별 likelihood
def lg_likelihood_i(x, y, beta, j) :
    p_hat = 0
    p = sigmoid(x[j], beta)
    p_hat += y[j]*np.log(p) + (1-y[j])*np.log(1-p)
    return p_hat
lg_likelihood_i(x, y, beta, 0)

-1.7354608880930498

## gradient Descent

In [602]:
# gradients 한 번 구하기
def get_gradients(x, y, beta):
    gradients = []
    
    for i in range(x[0].size) :
        gradient = 0            # 각 계수별 기울기
        for j in range(y.size) :
            p = sigmoid(x[j], beta)
            gradient += (y[j] - p)*x[j][i]
            
        gradient = gradient/y.size
        gradients.append(gradient)
    
    gradients = np.array(gradients)
    
    return gradients

gradients = np.array(get_gradients(x, y, beta))
gradients

array([-0.25616385, -0.11939429, -0.29846652])

In [649]:
def step(beta, gradients, stepsize=np.array([0.01,0.01,0.01])) :
    beta = beta + stepsize*gradients
    return beta

In [606]:
step(beta, gradients)

array([0.09236853, 0.69036544, 0.4906413 ])

In [659]:
def gradientDescent(x, y, beta, max_cycle = 200000, tolerance = 0.000001, stepsize=np.array([0.05,0.05,0.05])) :
    theta_0 = beta
    i = 0
    cost = lg_likelihood(x, y, theta_0)/y.size
    gradients = np.array([])
    while i < max_cycle:
        gradients = get_gradients(x, y, theta_0)
        theta = step(theta_0, gradients, stepsize)
        temp = theta_0 - theta
        theta_0 = theta
        
        if i % 1000 == 0:
            print(gradients)
            #print(theta_0)
            #print(theta)
            #print(np.abs(temp.sum()))
        if np.abs(temp.sum()) < tolerance :
            print("stop")
            break
        i += 1
    return theta_0

In [647]:
beta.sum()

1.0787823447303087

In [660]:
beta = gradientDescent(x, y, beta)
beta

[-0.28105306 -0.04337432 -0.2745462 ]
[-0.00683375  0.02426973 -0.02388329]
[-0.00376994  0.01178443 -0.01151959]
[-0.00236674  0.00713301 -0.00694437]
[-0.00160012  0.00473994 -0.00460212]
[-0.00112927  0.00331199 -0.00320966]
[-0.00081874  0.00238607 -0.00230924]
[-0.00060437  0.00175377 -0.00169564]
[-0.00045172  0.00130686 -0.00126263]
[-0.00034064  0.00098337 -0.00094957]
[-0.00025855  0.0007452  -0.00071929]
[-0.00019719  0.00056767 -0.00054776]
[-0.00015093  0.00043411 -0.00041878]
[-0.00011583  0.00033294 -0.00032113]
[-8.90811111e-05  2.55920132e-04 -2.46805093e-04]
[-6.86154614e-05  1.97047712e-04 -1.90008995e-04]
[-5.29149633e-05  1.51914112e-04 -1.46475418e-04]
[-4.08445459e-05  1.17234117e-04 -1.13029751e-04]
[-3.15497886e-05  9.05398363e-05 -8.72884826e-05]
[-2.43834469e-05  6.99647452e-05 -6.74496769e-05]
stop


array([-2.1101016 ,  4.52136689, -4.38941492])

In [661]:
lg_likelihood(x, y, beta) # 수렴한 최소 손실함수

-57.47849343202163